In [1]:
import getpass
import os
from dotenv import load_dotenv

load_dotenv()

os.environ["LANGSMITH_TRACING"] = "true"

if "LANGSMITH_API_KEY" not in os.environ:
    os.environ["LANGSMITH_API_KEY"] = getpass.getpass(
        prompt="Enter your LangSmith API Key (optional):"
    )

if "LANGSMITH_PROJECT" not in os.environ:
    os.environ["LANGSMITH_PROJECT"] = getpass.getpass(
        prompt="Enter your LangSmith Project Name (default = 'default'):"
    )
    if not os.environ.get("LANGSMITH_PROJECT"):
        os.environ["LANGSMITH_PROJECT"] = "default"

if not os.environ.get("COHERE_API_KEY"):
  os.environ["COHERE_API_KEY"] = getpass.getpass("Enter API key for Cohere: ")

if not os.environ.get("TAVILY_API_KEY"):
  os.environ["TAVILY_API_KEY"] = getpass.getpass("Enter API key for Tavily: ")


In [5]:
from langchain_community.tools.tavily_search import TavilySearchResults

search = TavilySearchResults(max_results=2)
search_results = search.invoke('What is weather in SF')
print(search_results)

tools = [search]

[{'title': 'Weather in San Francisco in June 2025 (California)', 'url': 'https://world-weather.info/forecast/usa/san_francisco/june-2025/', 'content': '*   [16 +68° +57°](https://world-weather.info/forecast/usa/san_francisco/14days/#2025-06-16)\n*   17 +70°\n+59°', 'score': 0.9521889}, {'title': 'San Francisco weather in June 2025 | Weather25.com', 'url': 'https://www.weather25.com/north-america/usa/california/san-francisco?page=month&month=June', 'content': '15°/12°](https://www.weather25.com/north-america/usa/california/san-francisco?page=day#date=2025-06-15)[Monday Jun 16 ![Image 13: Overcast](https://res.weather25.com/images/weather_icons/new/day/overcast.svg) 0 mm 15°/12°](https://www.weather25.com/north-america/usa/california/san-francisco?page=day#date=2025-06-16)[Tuesday Jun 17 ![Image 14: Sunny](https://res.weather25.com/images/weather_icons/new/day/sunny.svg) 0 mm [...] 16°/12°](https://www.weather25.com/north-america/usa/california/san-francisco?page=day#date=2025-06-17)[Wed

In [6]:
from langchain.chat_models import init_chat_model
from langchain_core.messages import HumanMessage, AIMessage

model = init_chat_model("command-r-plus", model_provider='cohere')
response = model.invoke([HumanMessage(content="hi!")])
response.content

'Hello! How can I help you today?'

In [7]:
model_with_tools = model.bind_tools(tools)

In [8]:
response = model_with_tools.invoke([HumanMessage('Hi')])
print(response.content)
print(response.tool_calls)

Hello! How can I help?
[]


In [9]:
response = model_with_tools.invoke([HumanMessage('What is weather in SF')])
print(response.content)
print(response.tool_calls)

I will search for the weather in San Francisco and write an answer based on what I find.
[{'name': 'tavily_search_results_json', 'args': {'query': 'weather in SF'}, 'id': 'tavily_search_results_json_rnb5ydyv33c7', 'type': 'tool_call'}]


In [10]:
from langgraph.prebuilt import create_react_agent

agent_executor = create_react_agent(model, tools)

In [12]:
response = agent_executor.invoke(
    {"messages": [HumanMessage('Hi')]}
    )
response

{'messages': [HumanMessage(content='Hi', additional_kwargs={}, response_metadata={}, id='c7ab0ee7-5654-4781-a7c1-ebea923010ae'),
  AIMessage(content='Hello! How can I help?', additional_kwargs={'id': 'f66a5c99-293c-4fdd-9acd-47c29bce7647', 'finish_reason': 'COMPLETE', 'content': 'Hello! How can I help?', 'token_count': {'input_tokens': 979.0, 'output_tokens': 33.0}}, response_metadata={'id': 'f66a5c99-293c-4fdd-9acd-47c29bce7647', 'finish_reason': 'COMPLETE', 'content': 'Hello! How can I help?', 'token_count': {'input_tokens': 979.0, 'output_tokens': 33.0}}, id='run--3474c26e-40ef-4df5-8bbd-2c069dbd191a-0', usage_metadata={'input_tokens': 979, 'output_tokens': 33, 'total_tokens': 1012})]}

In [11]:
response = agent_executor.invoke(
    {"messages": [HumanMessage('What is weather in SF')]}
    )
response

{'messages': [HumanMessage(content='What is weather in SF', additional_kwargs={}, response_metadata={}, id='fcd39d77-b06a-4f6d-8b67-5625f3e9cd9e'),
  AIMessage(content='I will search for the weather in San Francisco and write an answer based on what I find.', additional_kwargs={'id': '7e56d84e-4e8c-4f88-a8c0-1cfbc054fe46', 'finish_reason': 'TOOL_CALL', 'tool_plan': 'I will search for the weather in San Francisco and write an answer based on what I find.', 'tool_calls': [{'id': 'tavily_search_results_json_9xx8pc69s2j6', 'type': 'function', 'function': {'name': 'tavily_search_results_json', 'arguments': '{"query":"weather in SF"}'}}], 'token_count': {'input_tokens': 928.0, 'output_tokens': 69.0}}, response_metadata={'id': '7e56d84e-4e8c-4f88-a8c0-1cfbc054fe46', 'finish_reason': 'TOOL_CALL', 'tool_plan': 'I will search for the weather in San Francisco and write an answer based on what I find.', 'tool_calls': [{'id': 'tavily_search_results_json_9xx8pc69s2j6', 'type': 'function', 'function'

In [13]:
for step in agent_executor.stream(
    {"messages": [HumanMessage('What is weather in SF')]},
    stream_mode='values'
):
    step["messages"][-1].pretty_print()

================================ Human Message =================================

What is weather in SF
================================== Ai Message ==================================

I will search for the weather in San Francisco and write an answer based on the results.
Tool Calls:
  tavily_search_results_json (tavily_search_results_json_cw6hm7yj597v)
 Call ID: tavily_search_results_json_cw6hm7yj597v
  Args:
    query: weather in SF
================================= Tool Message =================================
Name: tavily_search_results_json

[{"title": "Monday, June 16, 2025. San Francisco, CA - Weather Forecast", "url": "https://weathershogun.com/weather/usa/ca/san-francisco/480/june/2025-06-16", "content": "San Francisco, California Weather: \n\nMonday, June 16, 2025\n\nDay 64°\n\nNight 54°\n\nPrecipitation 0 %\n\nWind 16 mph\n\nUV Index (0 - 11+)11\n\n[Tuesday](https://weathershogun.com/weather/usa/ca/san-francisco/480/june/2025-06-17) [...] Monday, June 16, 2025. San Franci

In [14]:
for step, metadata in agent_executor.stream(
    {"messages": [HumanMessage(content="whats the weather in sf?")]},
    stream_mode="messages",
):
    if metadata["langgraph_node"] == "agent" and (text := step.text()):
        print(text, end="|")

I| will| search| for| the| weather| in| San| Francisco| and| write| an| answer|.|It|'s| 6|4|°|F| in| San| Francisco| today|,| with| a| low| of| 5|4|°|F| tonight|.|

In [16]:
from langgraph.checkpoint.memory import MemorySaver

memory = MemorySaver()

In [17]:
agent_executor = create_react_agent(model, tools, checkpointer=memory)

In [18]:
config = {"configurable":{"thread_id":"abc123"}}

In [19]:
for chunk in agent_executor.stream(
    {"messages": [HumanMessage(content="hi I'm Bob")]},
    config
):
    print(chunk)
    print("-----")

{'agent': {'messages': [AIMessage(content='Hi Bob! How can I help?', additional_kwargs={'id': '23796ad2-67e2-48bf-bc38-160cce728420', 'finish_reason': 'COMPLETE', 'content': 'Hi Bob! How can I help?', 'token_count': {'input_tokens': 982.0, 'output_tokens': 35.0}}, response_metadata={'id': '23796ad2-67e2-48bf-bc38-160cce728420', 'finish_reason': 'COMPLETE', 'content': 'Hi Bob! How can I help?', 'token_count': {'input_tokens': 982.0, 'output_tokens': 35.0}}, id='run--4b221bf6-a16f-411d-865e-95ec3c6e264f-0', usage_metadata={'input_tokens': 982, 'output_tokens': 35, 'total_tokens': 1017})]}}
-----


In [20]:
for chunk in agent_executor.stream(
    {"messages": [HumanMessage(content="whats my name?")]}, config
):
    print(chunk)
    print("----")

{'agent': {'messages': [AIMessage(content='Your name is Bob.', additional_kwargs={'id': 'c0bd2efd-2ef5-40ac-b432-3bcf996cb3a3', 'finish_reason': 'COMPLETE', 'content': 'Your name is Bob.', 'token_count': {'input_tokens': 999.0, 'output_tokens': 29.0}}, response_metadata={'id': 'c0bd2efd-2ef5-40ac-b432-3bcf996cb3a3', 'finish_reason': 'COMPLETE', 'content': 'Your name is Bob.', 'token_count': {'input_tokens': 999.0, 'output_tokens': 29.0}}, id='run--e77760b1-6967-429a-a91c-1fe64e4c2705-0', usage_metadata={'input_tokens': 999, 'output_tokens': 29, 'total_tokens': 1028})]}}
----


In [21]:
config = {"configurable": {"thread_id": "xyz123"}}
for chunk in agent_executor.stream(
    {"messages": [HumanMessage(content="whats my name?")]}, config
):
    print(chunk)
    print("----")

{'agent': {'messages': [AIMessage(content="I don't know your name, I'm sorry.", additional_kwargs={'id': 'f45926dd-604f-4739-a919-817135dd82b9', 'finish_reason': 'COMPLETE', 'content': "I don't know your name, I'm sorry.", 'token_count': {'input_tokens': 980.0, 'output_tokens': 41.0}}, response_metadata={'id': 'f45926dd-604f-4739-a919-817135dd82b9', 'finish_reason': 'COMPLETE', 'content': "I don't know your name, I'm sorry.", 'token_count': {'input_tokens': 980.0, 'output_tokens': 41.0}}, id='run--4207ad72-185f-4ac9-b901-5a2946f806ec-0', usage_metadata={'input_tokens': 980, 'output_tokens': 41, 'total_tokens': 1021})]}}
----
